In [12]:
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go

from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)
import dash_canvas

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [13]:
# Start the app
app = Dash(__name__)

In [14]:
farm = pd.read_csv(r'./farm_mock.csv')
farm = farm.drop(['Unnamed: 0'], axis=1)
farm.head()

,x,y,manure_t,biogas_potential_m3
0,6.765843,52.314139,7431,2898090.0
1,6.837249,52.425084,7246,2825940.0
2,6.859361,52.386225,9639,3759210.0
3,6.753483,52.434382,9821,3830190.0
4,6.484077,52.243620,2732,1065480.0


In [15]:
# # -- Import and clean data (importing csv into pandas)
# # df = pd.read_csv("intro_bees.csv")
# df = pd.read_csv(r"./intro_bees.csv")

# df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
# df.reset_index(inplace=True)
# print(df[:5])

In [16]:
# ------------------------------------------------------------------------------
# Dash components
# App layout
app.layout = html.Div([

    # set title of web page 
    html.H1("Web Application Dashboards with Dash - K-means Clustering", style={'text-align': 'center'}),
    dcc.Markdown("Select the number of clusters (k):"),

    dcc.Dropdown(id="k-selector",
                 options=[
                     {'label':str(i), 'value': i} for i in range(2, 7)
                 ],
                #  options=[
                #      {"label": "3", "value": 3}, # connect drop down label to value, which are in interger mode for call back
                #      {"label": "4", "value": 4},
                #      {"label": "5", "value": 5},
                #      {"label": "6", "value": 6}],
                 multi=False,
                 value=3, # Default value
                 style={'width': "40%"}
                 ),

    # html.Div(id='output_container', children=[]),
    # html.Br(), # break line

    # graph 
    dcc.Graph(id='cluster-plot'),
    dash_canvas.DashCanvas(
        id='canvas',
        width=400,
        height=400
    )
])

In [32]:
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
# @app.callback(
#     [Output(component_id='output_container', component_property='children'), # output of map and output container
#      Output(component_id='my_bee_map', component_property='figure')],
#     [Input(component_id='slct_year', component_property='value')] # every input needs a def function for arg
# )

@app.callback(
    [Output('cluster-plot', 'figure'), Output('canvas', 'json_data')],
    Input('k-selector', 'value')
)

def update_graph(option_slctd): # 'value' in input goes as input to this def function
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy() # make a copy of the data used
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fig


In [33]:
# app.run_server(mode='inline')

In [37]:
app.run(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

2015
<class 'int'>


In [17]:
app.run_server(mode='external')

In [36]:
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run()

2015
<class 'int'>
